# Читаем обучающий датасет и делаем стратифицированное разбиение


dev датасет составляет примерно 5% от трейна. 

Для валидации отпилю 15% выборки, должно хватить. 

In [1]:
import numpy as np
import random

In [2]:
random.seed(17)
np.random.seed(17)

In [3]:
import json

with open('../data/train_dataset.json') as f:
    data = json.load(f)

Копипаста с ноутбука с EDA

In [4]:
from collections import Counter

import pandas as pd


def count_entities(sample):
    collapsed_entities = []
    saw_begin_before = False

    for label in sample['token_labels']:
        if label == 'O':
            saw_begin_before = False
        elif label[0] == 'B':
            saw_begin_before = True
            collapsed_entities.append(label[2:])
        elif label[0] == 'I':
            if saw_begin_before:
                continue
            else:
                raise ValueError("Found I-label without B-label before")
    
    c = dict(Counter(collapsed_entities))
    c['lang'] = sample['lang']
    return c


entity_counts = pd.DataFrame(
    [count_entities(sample) for sample in data]
).fillna(0)

Бить будем по текстам, а не по сущностям. Поэтому нам нужно привести все счетчики сущностей к бинарному виду

In [5]:
entity_names = ['GRP', 'PER', 'CW', 'PROD', 'CORP', 'LOC']

entity_counts[entity_names] = (entity_counts[entity_names] > 0).astype(int)

In [6]:
entity_counts.head()

,GRP,lang,PER,CW,PROD,CORP,LOC
0,1,RU-Russian,0,0,0,0,0
1,0,RU-Russian,1,0,0,0,0
2,1,RU-Russian,0,0,0,0,0
3,0,RU-Russian,0,1,0,0,0
4,0,RU-Russian,0,0,1,0,0


Стратицифированное разбиение из sklearn работает с одним столбцом "целевая переменная". Нам это не подходит, сделаю свою реализацию. 


Идеально провести разбиение, судя по всему, не получится, поэтому сделаем "перекос" максимально близким к тому, что наблюдается в train/dev

In [13]:
langmeans_diffs = pd.read_csv('../data/langmeans_diffs.csv')

In [14]:
langmeans_diffs.head()

,lang,GRP,PER,CW,PROD,CORP,LOC
0,BN-Bangla,0.009690,0.009673,0.009020,0.029134,0.010989,0.027149
1,DE-German,0.024020,0.023595,0.004469,0.023954,0.008701,0.033440
2,EN-English,0.000948,0.002688,0.024747,0.003913,0.033113,0.018039
3,ES-Spanish,0.003276,0.004322,0.000441,0.003358,0.006446,0.028456
4,FA-Farsi,0.000531,0.019951,0.012296,0.004739,0.002247,0.014788


['PROD', 'CORP', 'PER', 'CW', 'LOC', 'GRP']

In [29]:
langs = entity_counts['lang'].unique()


val_indices = []

for lang in langs:  # работаем с каждым языком по отдельности
    
    # частоты у сущностей, добавляемых первыми, искажаются сильнее
    # поэтому мы начинаем с наиболее "искаженных" сущностей для данного языка
    entity_names = langmeans_diffs.loc[langmeans_diffs['lang'] == lang, entity_names].T.iloc[:,0].sort_values(ascending=False).index.tolist()
    
    
    for entity in entity_names:  # и для каждой сущности
        total = entity_counts.loc[entity_counts['lang'] == lang, entity].sum()
        quota = int(total * 0.15)  # сколько всего текстов нужно отложить в валидацию
        already_added = entity_counts.loc[(entity_counts['lang'] == lang) & entity_counts.index.isin(val_indices), entity].sum()  # сколько текстов из уже добавленных в валидацию содержат эту сущность
        to_add = quota - already_added
        
        # Наблюдается проблема: из-за того, что в одном тексте могут быть несколько сущностей разного вида,
        # мы получаем сильное искажение соотношений для тех сущностей, которые мы добавили в начале. 
        # Чтобы ее сгладить, мы будем понижать вероятность взять в валидацию те тексты, в которых много сущностей разного вида. 
        # Просто линейно 
        
        probabilities = 1 / entity_counts.loc[(entity_counts['lang'] == lang) & (entity_counts[entity] == 1), entity_names].sum(axis=1)
        probabilities /= probabilities.sum()
        
        
        new_val_indices = np.random.choice(entity_counts.loc[(entity_counts['lang'] == lang) & (entity_counts[entity] == 1)].index, to_add, replace=False, p=probabilities)
        val_indices.extend(new_val_indices)

Проверяем корректность разбиения. Аналогично EDA (сейчас опять будет копипаста кода), убеждаемся, что по всем языкам и по всем сущностям мы сохранили их соотношение

In [30]:
entity_counts['type'] = 'train'
entity_counts.loc[val_indices, 'type'] = 'validation'

entity_counts.groupby(['lang', 'type'])[entity_names].mean()

PROD      CORP       PER        CW       LOC  \
lang       type                                                           
BN-Bangla  train       0.208333  0.169742  0.170357  0.140990  0.153367   
           validation  0.208551  0.169721  0.170157  0.140925  0.153578   
DE-German  train       0.170610  0.184743  0.255720  0.202858  0.200984   
           validation  0.160449  0.171280  0.259928  0.180907  0.212194   
EN-English train       0.167631  0.179593  0.261298  0.204457  0.207193   
           validation  0.152590  0.201195  0.244622  0.229084  0.212351   
ES-Spanish train       0.171091  0.169378  0.240134  0.216315  0.212034   
           validation  0.161843  0.174480  0.258459  0.195271  0.232369   
FA-Farsi   train       0.171417  0.180194  0.220893  0.199767  0.216777   
           validation  0.164124  0.169485  0.262268  0.194639  0.222680   
HI-Hindi   train       0.199215  0.175497  0.155783  0.147775  0.158478   
           validation  0.198790  0.176750  0.158168  0.146500  0.159464   
KO-Korean  train       0.162979  0.193129  0.226862  0.228732  0.236756   
           validation  0.167614  0.199269  0.257305  0.207792  0.249188   
NL-Dutch   train       0.169468  0.165266  0.229692  0.188920  0.232415   
           validation  0.165441  0.182190  0.231209  0.203840  0.222631   
RU-Russian train       0.166615  0.171116  0.219541  0.189042  0.210306   
           validation  0.164872  0.183099  0.212096  0.198012  0.195112   
TR-Turkish train       0.174136  0.164583  0.232621  0.201243  0.218408   
           validation  0.166598  0.169072  0.214845  0.206186  0.226392   
ZH-Chinese train       0.275776  0.227077  0.115923  0.288986  0.294223   
           validation  0.264858  0.209015  0.100917  0.305146  0.305146   

                            GRP  
lang       type                  
BN-Bangla  train       0.157211  
           validation  0.157068  
DE-German  train       0.197236  
           validation  0.208183  
EN-English train       0.207897  
           validation  0.184462  
ES-Spanish train       0.193664  
           validation  0.183449  
FA-Farsi   train       0.182835  
           validation  0.169485  
HI-Hindi   train       0.184814  
           validation  0.182800  
KO-Korean  train       0.195388  
           validation  0.177760  
NL-Dutch   train       0.192810  
           validation  0.175654  
RU-Russian train       0.181903  
           validation  0.182270  
TR-Turkish train       0.189748  
           validation  0.199588  
ZH-Chinese train       0.039944  
           validation  0.043079

И так же, как и в EDA, посмотрим на разницу средних

In [32]:
val_langmeans = entity_counts.groupby(['lang', 'type'])[entity_names].mean().reset_index(level=1)
val_langmeans_diffs = (val_langmeans.loc[val_langmeans['type'] == 'train', entity_names] - val_langmeans.loc[val_langmeans['type'] == 'validation', entity_names]).abs()
val_langmeans_diffs.max()

PROD    0.015041
CORP    0.021602
PER     0.041375
CW      0.024627
LOC     0.020335
GRP     0.023435
dtype: float64

В целом, разбиение получилось довольно близким, хотя местами есть довольно заметная разница

In [47]:
(langmeans_diffs.set_index('lang') - val_langmeans_diffs).abs()

,CORP,CW,GRP,LOC,PER,PROD
lang,,,,,,
BN-Bangla,0.010968,0.008954,0.009547,0.026938,0.009474,0.028916
DE-German,0.004762,0.017482,0.013073,0.022229,0.019387,0.013794
EN-English,0.011511,0.000120,0.022487,0.012882,0.013988,0.011128
ES-Spanish,0.001344,0.020603,0.006939,0.008121,0.014003,0.005890
FA-Farsi,0.008463,0.007168,0.012819,0.008884,0.021424,0.002555
HI-Hindi,0.008183,0.006307,0.001524,0.006392,0.006472,0.010425
KO-Korean,0.007977,0.018080,0.013829,0.003826,0.005958,0.015390
NL-Dutch,0.003853,0.000023,0.008342,0.000616,0.010917,0.009797
RU-Russian,0.001239,0.005677,0.002844,0.014352,0.006578,0.003167


В принципе, удалось разбить немного получше, чем в train/dev разбиении от оргов. 

# Сохраняем разбиение

Тут есть сложность, связанная с тем, что датасет в json может сгенерироваться немного по-разному. 

Поэтому сперва делаем сортировку всего датасета по длине, потом сохраняем измененные индексы. Это должно гарантировать, что тренировочная и валидационная выборки у всех будут одинаковыми. 

In [48]:
entity_counts.shape

(168300, 8)

In [49]:
entity_counts['text'] = [' '.join(d['tokens']) for d in  data]

In [50]:
entity_counts.head()

,GRP,lang,PER,CW,PROD,CORP,LOC,type,text
0,1,RU-Russian,0,0,0,0,0,train,российская экологическая партия «зелёные» — до...
1,0,RU-Russian,1,0,0,0,0,train,также посещал два семинара бартольда кёйкена п...
2,1,RU-Russian,0,0,0,0,0,train,в 1999 — 2006 играла за национальную сборную с...
3,0,RU-Russian,0,1,0,0,0,train,« джульетта » — кинофильм 2016 года испанского...
4,0,RU-Russian,0,0,1,0,0,train,мякоть спелого плода съедобна в свежем виде и ...


In [54]:
entity_counts.sort_values(by='text', inplace=True)
entity_counts.reset_index(inplace=True)

In [56]:
new_val_indices = entity_counts[entity_counts['type'] == 'validation'].index

In [57]:
new_val_indices

Int64Index([     0,      7,     16,     20,     21,     33,     40,     51,
                54,     56,
            ...
            168238, 168239, 168240, 168242, 168243, 168267, 168277, 168287,
            168291, 168296],
           dtype='int64', length=26745)

In [61]:
with open('../data/val_indices.txt', 'w') as f:
    f.writelines('\n'.join(new_val_indices.astype(str).tolist()))